Anexo R

# Utilizando Google Colab para realizar petición (Query) a Red Neuronal alojada en IBM Watson.

#### Elaborado por: Ricardo Niño de Rivera Barrón

#### Ingeniería Biónica

#### Trabajo Terminal II

En esta libreta interactiva se hace una demostración de acceso al modelo deplegado en IBM Watson desde el entorno de Google Colab.

Instalamos biblioteca para entablar comunicación con MEGA.



In [1]:
!pip install mega.py

     |████████████████████████████████| 13.7MB 300kB/s 


In [4]:
# De la biblioteca mega importamos el método Mega
from mega import Mega

In [5]:
# Instanciando Mega en el objeto mega
mega = Mega()

Inicamos sesión en MEGA con cuenta temporal y anónima

In [6]:
# Log in en la cuenta de MEGA con cuenta temporal anónima
m = mega.login()

In [7]:
# Descargando el archivo X_train.npy
m.download_url('https://mega.nz/file/oAZT1IYJ#HpIvnnR50IH6N2F9wfdPgNTgLPpAwhEBpTl3_yAxLFc')

PosixPath('X_train.npy')

In [8]:
# Mostrando archivos en el directorio de trabajo del entorno
!ls

sample_data  X_train.npy


In [9]:
# Descargando el archivo X_test.npy
m.download_url('https://mega.nz/file/tMIzDQbK#MLFeDRu5D4dq2lRqvPcxIZkuTtVKH2azBRrhSzndqME')

PosixPath('X_test.npy')

In [10]:
# Imprimiendo el directorio de trabajo
!pwd

/content


Descargando el archivo de salida Y_train y Y_test

In [11]:
# Descargando Y_train2.npy
m.download_url('https://mega.nz/file/tdohXarb#SRrt31GIT7H6gioCVm2KYF-sH6a8PHK8Z7Tr76oZb4I')

PosixPath('Y_train2.npy')

In [12]:
!ls

sample_data  X_test.npy  X_train.npy  Y_train2.npy


In [13]:
# Descargando Y_test2.npy
m.download_url('https://mega.nz/file/Vc5TBSbY#D8pbUvpA3l2V0PJxCu7Meg8WJ3wPpT6X2jEoDsleCWw')

PosixPath('Y_test2.npy')

In [14]:
!ls

sample_data  X_test.npy  X_train.npy  Y_test2.npy  Y_train2.npy


Leyendo lo archivos con numpy

In [15]:
import numpy as np

In [16]:
X_train = np.load('X_train.npy')
X_train.shape

(3337, 17)

In [17]:
X_test = np.load('X_test.npy')
X_test.shape

(832, 17)

In [18]:
Y_train = np.load('Y_train2.npy')
Y_train.shape

(3337,)

In [19]:
Y_test = np.load('Y_test2.npy')
Y_test.shape

(832,)

### Descargando el modelo

In [20]:
# Descargando archivo del modelo completo PCA_rnet_FULL.h5
m.download_url('https://mega.nz/file/Jdw0xRAb#niBMdBeTC0ZcPWhQz-pavMPrwLXru9tt6aVBOvO2u6s')

PosixPath('PCA_rnet_FULL.h5')

In [21]:
!ls

PCA_rnet_FULL.h5  X_test.npy   Y_test2.npy
sample_data	  X_train.npy  Y_train2.npy


In [22]:
# Importando tensorflow
import tensorflow as tf
import tensorflow_addons as tfa

In [23]:
rnet=tf.keras.models.load_model('PCA_rnet_FULL.h5')

## PCA

Creando el modelo que reduce dimensionalmente X_test a partir de los datos X_train.

In [24]:
# Importando el método PCA
from sklearn.decomposition import PCA

In [25]:
# Instanciando PCa en un objeto que transforme los datos a 3 dimensiones
pca = PCA(n_components=3)

In [26]:
# Para X_train
X_train_pca = pca.fit(X_train).transform(X_train)

In [27]:
# Para X_test
# Nótese que se utiliza X_train para ajustar el pca
X_test_pca = pca.fit(X_train).transform(X_test)

In [28]:
# Comprobación con un ejemplo cualesquiera

#Leyendo ejemplo
ejemplo_test = X_test[5,:]

#Expandiendo una dimension
ejemplo_test = np.expand_dims(ejemplo_test,axis=0)

#Transormando el ejemplo
prueba1 = pca.fit(X_train).transform(ejemplo_test)
print(prueba1)
print(X_test_pca[5,:])

[[-18.52513566  21.87689658  -5.58046172]]
[-18.52513566  21.87689658  -5.58046172]


In [29]:
# Función para retornar vector de carcterísticas después de aplicar PCA
def PCA_transform(vector):
    
    #Expandiendo una dimension
    vector = np.expand_dims(vector,axis=0)
    
    #Transformando el ejemplo
    vector_pca = pca.fit(X_train).transform(vector)
    
    return vector_pca

In [30]:
#Prueba 2
ejemplo_test2 = X_test[515,:]
#Transformando el ejemplo
prueba2 = PCA_transform(ejemplo_test2)
print(prueba2)
print(X_test_pca[515,:])

[[-22.83520649   5.3994245    3.07702366]]
[-22.83520649   5.3994245    3.07702366]


# Realizando Petición (Query)

In [31]:
# Definimos el arreglo que enviaremos para realizar la petición al modelo alojado en IBM WATOSN CLOUD
score_0 = PCA_transform(X_test[515,:]).tolist()

In [32]:
# Importando bibliotecas necesarias para realizar petcición
import requests
import json

In [33]:
# Estableciendo datos de la aplicación WEB en IBM WATSON CLOUD
# Y generando un token temporal para cceder a la aplicación
API_KEY = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
token_response = requests.post('https://iam.ng.bluemix.net/identity/token', data={"apikey": API_KEY, "grant_type": 'urn:ibm:params:oauth:grant-type:apikey'})
mltoken = token_response.json()["access_token"]

header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

In [34]:
# Definimos el diccionario para la petición
scoring_payload = {"input_data": [{"values": [score_0[0]]}]}

In [35]:
response_scoring = requests.post('https://us-south.ml.cloud.ibm.com/ml/v4/deployments/XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX/predictions?version=2020-12-04', json=scoring_payload, headers={'Authorization': 'Bearer ' + mltoken})
print("Scoring response")
print(response_scoring.json())

Scoring response
{'predictions': [{'id': 'dense_3', 'fields': ['prediction', 'prediction_classes', 'probability'], 'values': [[[0.017080986872315407], [0], [0.017080986872315407]]]}]}


# Imprimiendo la probabilidad de cáncer mamario

In [36]:
# Modelo en IBM WATSON CLOUD
response_scoring.json()['predictions'][0]['values'][0][0][0]

0.017080986872315407

In [37]:
# Modelo "local"
rnet.predict(score_0)[0][0]

0.017080992

**NOTAS:** la probabilidad mostrada es un valor entre 0 y 1.
